In [1]:
import pandas as pd
import difflib

# Load the Excel file
file_path = './Downloads/BatchingTool_DataProcessing_837SubmissionSupports_TCM_202406_20241007v6_SignedOff.xlsm'

df1 = pd.read_excel(file_path, sheet_name='RawDataDetails')

In [2]:
masterdf = pd.read_excel("./Downloads/tcm/Master_db.xlsx")

In [3]:
masterdf_selected = masterdf[['MEMBER ID', 'LAST NAME', 'FIRST NAME', 'MedicaidID', "DOB"]]

In [5]:
df1[['FIRST NAME', 'LAST NAME']] = df1['MEMBER NAME'].str.split(n=1, expand=True)

In [6]:
df1["FIRST NAME"] = df1["FIRST NAME"].str.upper()
# Clean and reduce last name to the last word only
df1['LAST NAME'] = (
    df1['LAST NAME']
    .str.strip()  # remove leading/trailing spaces
    .str.replace(r'[\t\n\r\f\v]+', '', regex=True)  # remove escape characters
    .str.split()  # split into words
    .str[-1]  # take only the last word
    .str.upper()  # convert to uppercase
)

In [7]:
df1_selected = df1[["MEMBER NAME", "MEMBER ID","DATE OF SERVICE","MEMBER DOB", "MEDICAID ID", "FIRST NAME", "LAST NAME","TCM HOURS","TCM UNITS","ICD 10","CPT CODE"]]

In [8]:
def correct_member_info(df1_selected, masterdf):
    # Keep track of original records
    result_df = df1_selected.copy()
    
    # Create a mapping dictionary from masterdf
    master_mapping = {}
    for _, row in masterdf.iterrows():
        dob = row['DOB']
        if dob not in master_mapping:
            master_mapping[dob] = {
                'FIRST NAME': row['FIRST NAME'],
                'LAST NAME': row['LAST NAME'],
                'MedicaidID': row['MedicaidID']
            }
    
    def string_similarity(str1, str2):
        if pd.isna(str1) or pd.isna(str2):
            return 0
        return difflib.SequenceMatcher(None, str1.lower(), str2.lower()).ratio()
    
    # Process each row in original dataframe
    for idx, row in result_df.iterrows():
        dob = row['MEMBER DOB']
        if dob in master_mapping:
            master_info = master_mapping[dob]
            
            # Check name similarity
            first_name_match = string_similarity(row['FIRST NAME'], master_info['FIRST NAME'])
            last_name_match = string_similarity(row['LAST NAME'], master_info['LAST NAME'])
            
            if (first_name_match + last_name_match) / 2 > 0.8:
                result_df.at[idx, 'FIRST NAME'] = master_info['FIRST NAME']
                result_df.at[idx, 'LAST NAME'] = master_info['LAST NAME']
                result_df.at[idx, 'MEDICAID ID'] = master_info['MedicaidID']
    
    return result_df

# Apply the correction
corrected_df = correct_member_info(df1_selected, masterdf)

# Verify the counts
print(f"Original record count: {len(df1_selected)}")
print(f"Corrected record count: {len(corrected_df)}")

# Show some statistics about the corrections
changes = (df1_selected != corrected_df).sum()
print("\nNumber of changes made in each column:")
print(changes)

# Check for any remaining duplicate Medicaid IDs
duplicates = corrected_df[corrected_df.duplicated(subset=['MEDICAID ID'], keep=False)]
if not duplicates.empty:
    print("\nRecords with duplicate Medicaid IDs:")
    print(duplicates)

Original record count: 671
Corrected record count: 671

Number of changes made in each column:
MEMBER NAME         0
MEMBER ID           0
DATE OF SERVICE     0
MEMBER DOB          0
MEDICAID ID        20
FIRST NAME         52
LAST NAME          55
TCM HOURS           0
TCM UNITS           0
ICD 10              0
CPT CODE            0
dtype: int64

Records with duplicate Medicaid IDs:
            MEMBER NAME  MEMBER ID DATE OF SERVICE MEMBER DOB MEDICAID ID  \
0    Alan M \tStenglein        838      2025-01-06 1953-10-20     S527787   
1    Alan M \tStenglein        838      2025-01-30 1953-10-20     S527787   
2    Alan M \tStenglein        838      2025-01-30 1953-10-20     S527787   
3    Alan M \tStenglein        838      2025-01-31 1953-10-20     S527787   
4     Albert \tArguello        836      2025-01-22 1952-05-14     J208643   
..                  ...        ...             ...        ...         ...   
666       Walter \tBaum        751      2025-01-19 1964-07-01     O388696

In [9]:
corrected_df.to_csv('./correctedv1.csv')

In [15]:
# Group by Medicaid ID and Date of Service, then sum up TCM Hours and TCM Units
grouped_df = corrected_df.groupby(['MEDICAID ID', 'DATE OF SERVICE']).agg({
    'TCM HOURS': 'sum',
    'TCM UNITS': 'sum',
    'FIRST NAME': 'first',  # Keep first occurrence of these fields
    'LAST NAME': 'first',
    'MEMBER ID': 'first',
    'MEMBER DOB': 'first',
    'ICD 10': 'first',
    'CPT CODE': 'first'
}).reset_index()

# Sort by Medicaid ID and Date of Service
grouped_df = grouped_df.sort_values(['MEDICAID ID', 'DATE OF SERVICE'])

print("Records after combining same-day services:")

Records after combining same-day services:


In [25]:
# First create static values dictionary
static_values = {
    'npi': '1184543043',
    'location_id': 'FOCUSCARE', 
    'cpt_proc_code': 'T2023',
    'cpt_modifier': 'U1',
    'rate': 26.75
}

# Add static and calculated columns to grouped_df
grouped_df['[Claim Type]'] = 'Professional'
grouped_df['[Payer]'] = 'Title XIX Payer'
grouped_df['[Billing Provider ID]'] = static_values['npi']
grouped_df['[ID Type]'] = 'NPI'
grouped_df['[Provider Name]'] = 'FOCUS CARE SOLUTIONS INC.'
grouped_df['[Location]'] = static_values['location_id']
grouped_df['[Taxonomy]'] = '251B00000X'
grouped_df['[Taxonomy Descriptions]'] = 'Case Management'
grouped_df['[Transport Certification]'] = 'No'
grouped_df['[Prov Signature on File]'] = 'Yes'
grouped_df['[Diagnosis Type]'] = 'ICD-10-CM'
grouped_df['[From Date]'] = pd.to_datetime(grouped_df['DATE OF SERVICE'])
grouped_df['[To Date]'] = pd.to_datetime(grouped_df['DATE OF SERVICE'])
grouped_df['[Place Of Service]'] = '12'
grouped_df['[Procedure Code]'] = static_values['cpt_proc_code']
grouped_df['[Modifiers]'] = static_values['cpt_modifier']
grouped_df['[Diagnosis Pointers]'] = '1'
grouped_df['[Unit Type]'] = 'Unit'
grouped_df['[Rate]'] = static_values['rate']

# Add calculated columns
grouped_df['[Units]'] = grouped_df['TCM UNITS']  # Assuming this maps to TCM UNITS
grouped_df['[Hours]'] = grouped_df['TCM HOURS']  # Assuming this maps to TCM HOURS
grouped_df['[Charge Amount]'] = grouped_df['[Units]'] * grouped_df['[Rate]']
grouped_df['[Check1_ReconUnits]'] = (grouped_df['[Hours]'] * 4) - grouped_df['[Units]']
grouped_df['[Check1_ReconAmount]'] = (grouped_df['[Units]'] * grouped_df['[Rate]']) - grouped_df['[Charge Amount]']

# # Display the first few rows to verify
# print("\nFirst few rows of updated grouped_df:")
# print(grouped_df.head())

# # Display column names to verify all were added
# print("\nAll columns in updated grouped_df:")
# print(grouped_df.columns.tolist())

In [27]:
grouped_df.to_csv('./final_batch.csv')

In [16]:
df2 = pd.read_excel(file_path, sheet_name='Prep837')

In [22]:
# # Get unique Medicaid IDs from both dataframes
# medicaid_ids_df2 = set(df2['MEDICAID ID'].unique())
# medicaid_ids_grouped = set(grouped_df['MEDICAID ID'].unique())

# # Find extra Medicaid IDs in df2 (not in grouped_df)
# extra_in_df2 = medicaid_ids_df2 - medicaid_ids_grouped

# # Find extra Medicaid IDs in grouped_df (not in df2)
# extra_in_grouped = medicaid_ids_grouped - medicaid_ids_df2

# print("Medicaid IDs in df2 but not in grouped_df:")
# print(extra_in_df2)
# print("\nMedicaid IDs in grouped_df but not in df2:")
# print(extra_in_grouped)

# # Show counts
# print(f"\nTotal unique Medicaid IDs in df2: {len(medicaid_ids_df2)}")
# print(f"Total unique Medicaid IDs in grouped_df: {len(medicaid_ids_grouped)}")
# print(f"Number of extra IDs in df2: {len(extra_in_df2)}")
# print(f"Number of extra IDs in grouped_df: {len(extra_in_grouped)}")